In [1]:
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio
import numpy as np
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os # interface with underlying OS that python is running on
import sys

In [2]:
# LOAD IN FILE
#main_dir = 'D:/Makaleler/makale19 speech recog/ravdes/ravdes/Audio_Speech_Actors_01-24/'
main_dir = "D:/Makaleler/makale19 speech recog/emodb/wav/"
#sub_dir = os.listdir(main_dir)
x, sr = librosa.load(main_dir+'/03a01Wa.wav')
# PLAY any AUDIO FILE
#librosa.output.write_wav('MaleNeutral.wav', x, sr)
Audio(data=x, rate=sr)

In [25]:
"""
def extract_feature(file_name, offst=0.5):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast',offset=offst)
    
    stft = np.abs(librosa.stft(X))
    chroma_cq = librosa.feature.chroma_cqt(y=X, sr=sample_rate)
    cqt=np.mean(chroma_cq,axis=1)
    
    #chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate,n_fft=2048).T,axis=0)
    #chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate,n_mels=128,fmax=8000).T,axis=0)
    #mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,cqt,mel,contrast,tonnetz
"""
def extract_feature(file_name, offst=0.5):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast',offset=offst)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate,n_fft=2048).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate,n_mels=128,fmax=8000).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

def extract_featurev2(file_name):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast', sr=None)   
    #mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=64).T, axis=0)
    C = np.abs(librosa.cqt(X, sr=sr, fmin=librosa.note_to_hz('C2'), n_bins=60))
    #d=librosa.amplitude_to_db(C)
    d=librosa.power_to_db(C)
    cqt = np.mean(d,axis=1)
    return cqt

In [26]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [28]:
a= []
b= []
c= []
d= []
e= []
#from warnings import warn
import warnings
# run block of code and catch warnings
with warnings.catch_warnings():
	# ignore all caught warnings
	warnings.filterwarnings("ignore")
	# execute code that will generate warnings

X4= pd.DataFrame()
cc=0
for root, dirs, files in os.walk(main_dir, topdown=False):
    for f in files:
        #print(os.path.join(root, f))
        #print(f)
        a,b,c,d,e=extract_feature(os.path.join(root, f))
        #a=extract_feature(main_dir + i + '/' + f)
        tot= []
        for x in a:tot.append(x)
        for x in b:tot.append(x)
        for x in c:tot.append(x)
        for x in d:tot.append(x)
        for x in e:tot.append(x)
        
        #X4[cc]=pd.DataFrame([tot])
        X4[cc]=np.asarray(tot)
        cc+=1;
        #print(tot)

In [29]:
len(a),len(b),len(c),len(d),len(e),

(40, 12, 128, 7, 6)

In [30]:
X6= pd.DataFrame(X4.T)

In [31]:
type(X6), X6.shape

(pandas.core.frame.DataFrame, (535, 193))

In [32]:
def decompose_emodb():
        #EMODB_PATH = '/kaggle/input/berlin-database-of-emotional-speech-emodb/wav'
        emotion = []
        emov2 = []
        path = []
        gender = []  
        
        for root, dirs, files in os.walk(main_dir):
            for name in files:
                #print(name)
                if name[0:2] in ['03','10','11','12','15']: #'0310111215':  # MALE
                    if name[5] == 'W':  # Ärger (Wut) -> Angry
                        emotion.append('male_angry')
                        emov2.append(1)
                    elif name[5] == 'L':  # Langeweile -> Boredom
                        emotion.append('male_bored')
                        emov2.append(2)
                    elif name[5] == 'E':  # Ekel -> Disgusted
                        emotion.append('male_disgust')
                        emov2.append(4)
                    elif name[5] == 'A':  # Angst -> Angry
                        emotion.append('male_fear')
                        emov2.append(5)
                    elif name[5] == 'F':  # Freude -> Happiness
                        emotion.append('male_happy')
                        emov2.append(6)
                    elif name[5] == 'T':  # Trauer -> Sadness
                        emotion.append('male_sad')
                        emov2.append(7)
                    elif name[5] == 'N':
                        emotion.append('male_neutral')
                        emov2.append(3)
                    else:
                        emotion.append('unknown')
                        emov2.append(0)
                        #print(name)
                    gender.append(0)
                else:
                    if name[5] == 'W':  # Ärger (Wut) -> Angry
                        emotion.append('female_angry')
                        emov2.append(1)
                    elif name[5] == 'L':  # Langeweile -> Boredom
                        emotion.append('female_bored')
                        emov2.append(2)
                    elif name[5] == 'E':  # Ekel -> Disgusted
                        emotion.append('female_disgust')
                        emov2.append(4)
                    elif name[5] == 'A':  # Angst -> Angry
                        emotion.append('female_fear')
                        emov2.append(5)
                    elif name[5] == 'F':  # Freude -> Happiness
                        emotion.append('female_happy')
                        emov2.append(6)
                    elif name[5] == 'T':  # Trauer -> Sadness
                        emotion.append('female_sad')
                        emov2.append(7)
                    elif name[5] == 'N':
                        emotion.append('female_neutral')
                        emov2.append(3)
                    else:
                        emotion.append('unknown')
                        emov2.append(0)
                    gender.append(1)
                #path.append(os.path.join( main_dir, name)) #"-", name)) #
                path.append( name)

        emodb_df = pd.DataFrame(emotion, columns=['labels'])
        emodb_emo1 = pd.DataFrame(emov2, columns=['emo'])
        emodb_df['source'] = 'EMODB'
        gender_df = pd.DataFrame(gender, columns=['gender'])
        emodb_df1 = pd.concat([emodb_df, emodb_emo1, gender_df, pd.DataFrame(path, columns=['path'])], axis=1)
        
        return emodb_df1

In [33]:
dfEmo=decompose_emodb()

In [34]:
dfEmo[210:220]

,labels,source,emo,gender,path
210,male_fear,EMODB,5,0,11a07Ac.wav
211,male_bored,EMODB,2,0,11a07Ld.wav
212,male_sad,EMODB,7,0,11a07Ta.wav
213,male_angry,EMODB,1,0,11a07Wc.wav
214,male_fear,EMODB,5,0,11b01Ab.wav
215,male_disgust,EMODB,4,0,11b01Eb.wav
216,male_happy,EMODB,6,0,11b01Fc.wav
217,male_bored,EMODB,2,0,11b01Lb.wav
218,male_neutral,EMODB,3,0,11b01Nc.wav
219,male_angry,EMODB,1,0,11b01Wd.wav


In [35]:
dfEmo['path'][1]

'03a01Nc.wav'

In [36]:
dfEmo.loc[dfEmo['emo']==0]
#df.loc[df['col1'] == value]

,labels,source,emo,gender,path


In [37]:
X7=X6.copy()

In [38]:
X7

,0,1,2,3,4,5,6,7,8,9,...,183,184,185,186,187,188,189,190,191,192
0,-291.023743,122.103310,-44.846886,59.766186,-24.904373,23.310034,-18.901543,-4.398162,-2.784216,-9.171474,...,14.751550,17.491031,16.934180,44.290817,0.010006,0.012283,-0.032002,0.024771,0.008110,0.001527
1,-275.305420,138.871933,-33.697144,49.522625,-7.244414,14.749916,-5.809273,-8.138051,-7.884721,-2.113137,...,14.512857,17.552152,19.170462,42.127514,-0.001838,0.006182,-0.084598,0.109258,-0.025841,-0.003582
2,-226.691788,95.119919,-73.665314,41.528965,-27.266602,11.262508,-13.882312,-9.181475,-10.468313,-10.647150,...,16.132340,18.235059,19.369836,43.109701,0.000832,-0.006184,-0.034412,-0.076670,0.011311,-0.001623
3,-274.819733,100.031342,-37.423729,36.635162,-22.137280,17.577888,-11.324630,-2.312308,-11.398092,-17.833254,...,15.813201,17.616387,17.451906,46.304244,0.039056,-0.017722,-0.080997,-0.049779,0.015512,-0.007105
4,-253.606750,150.312592,-25.341574,45.506142,-0.686996,5.291062,1.370816,-1.645677,-12.028738,-10.254305,...,14.599663,14.796250,19.763224,39.307039,0.004677,-0.009056,-0.025911,-0.049394,0.005211,0.028331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,-256.870941,106.805382,-42.942406,76.135025,-23.356741,9.561186,-15.960503,-6.739610,-2.011499,-10.854325,...,16.838264,17.836653,19.849010,44.480497,0.013730,-0.044924,0.036141,-0.103874,0.014027,0.004247
531,-247.077347,92.381683,-25.850836,57.561165,-6.191659,18.602076,-3.402066,-4.480618,3.263244,-6.546418,...,15.285443,17.030423,18.877839,45.172986,0.022582,0.051643,0.111258,0.034694,0.032592,-0.023957
532,-288.360962,89.563797,-22.478836,60.567955,-0.328118,19.141325,0.582287,1.187719,4.120030,-6.690523,...,14.401903,15.672472,17.697322,45.060143,0.064397,0.045831,-0.014433,0.036813,-0.002724,-0.039336
533,-330.462524,68.713730,-88.104507,64.599174,-49.094189,-2.239225,-19.608362,-12.148569,-5.809820,-29.100821,...,19.919413,19.620037,21.775347,42.181517,0.004429,0.004515,-0.046787,-0.008189,-0.011627,0.014819


In [39]:
X7 = pd.concat([X7,dfEmo['gender'],dfEmo['emo']],axis=1)

In [40]:
X7

,0,1,2,3,4,5,6,7,8,9,...,185,186,187,188,189,190,191,192,gender,emo
0,-291.023743,122.103310,-44.846886,59.766186,-24.904373,23.310034,-18.901543,-4.398162,-2.784216,-9.171474,...,16.934180,44.290817,0.010006,0.012283,-0.032002,0.024771,0.008110,0.001527,0,6
1,-275.305420,138.871933,-33.697144,49.522625,-7.244414,14.749916,-5.809273,-8.138051,-7.884721,-2.113137,...,19.170462,42.127514,-0.001838,0.006182,-0.084598,0.109258,-0.025841,-0.003582,0,3
2,-226.691788,95.119919,-73.665314,41.528965,-27.266602,11.262508,-13.882312,-9.181475,-10.468313,-10.647150,...,19.369836,43.109701,0.000832,-0.006184,-0.034412,-0.076670,0.011311,-0.001623,0,1
3,-274.819733,100.031342,-37.423729,36.635162,-22.137280,17.577888,-11.324630,-2.312308,-11.398092,-17.833254,...,17.451906,46.304244,0.039056,-0.017722,-0.080997,-0.049779,0.015512,-0.007105,0,6
4,-253.606750,150.312592,-25.341574,45.506142,-0.686996,5.291062,1.370816,-1.645677,-12.028738,-10.254305,...,19.763224,39.307039,0.004677,-0.009056,-0.025911,-0.049394,0.005211,0.028331,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,-256.870941,106.805382,-42.942406,76.135025,-23.356741,9.561186,-15.960503,-6.739610,-2.011499,-10.854325,...,19.849010,44.480497,0.013730,-0.044924,0.036141,-0.103874,0.014027,0.004247,1,2
531,-247.077347,92.381683,-25.850836,57.561165,-6.191659,18.602076,-3.402066,-4.480618,3.263244,-6.546418,...,18.877839,45.172986,0.022582,0.051643,0.111258,0.034694,0.032592,-0.023957,1,7
532,-288.360962,89.563797,-22.478836,60.567955,-0.328118,19.141325,0.582287,1.187719,4.120030,-6.690523,...,17.697322,45.060143,0.064397,0.045831,-0.014433,0.036813,-0.002724,-0.039336,1,7
533,-330.462524,68.713730,-88.104507,64.599174,-49.094189,-2.239225,-19.608362,-12.148569,-5.809820,-29.100821,...,21.775347,42.181517,0.004429,0.004515,-0.046787,-0.008189,-0.011627,0.014819,1,1


In [41]:
X7.to_csv("featureEmodb_Ek.csv",index=False,header=True)
#audio_df.to_csv("audioELM.csv",index=False,header=True)

In [19]:
dir ='D:/codes/datasets/ravdes/'
X7.to_excel(dir+"featureEmodbEk.xlsx",index=False) 

In [18]:
import os
data = pd.read_csv('featureShortc128.csv')
audio_df = pd.read_csv('audio_df.csv')

In [19]:
data

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,0.1,0.2
0,0.000002,0.000066,0.004286,0.024769,0.037116,0.020915,0.004085,0.022000,0.029166,0.024192,...,0.000002,7.983671e-07,4.238818e-07,4.620670e-07,3.084141e-07,7.357968e-08,1.610563e-08,1.311767e-09,1,1
1,0.000002,0.000093,0.004000,0.021978,0.060232,0.034005,0.003991,0.012792,0.034489,0.048722,...,0.000004,2.326391e-06,1.393828e-06,1.162513e-06,3.586766e-07,1.103367e-07,1.895200e-08,1.571617e-09,1,1
2,0.000004,0.000072,0.004056,0.025928,0.058986,0.018838,0.004335,0.025710,0.031794,0.059034,...,0.000007,5.198676e-06,4.122103e-06,2.653052e-06,1.764746e-06,3.578079e-07,5.847575e-08,6.194315e-09,1,1
3,0.000003,0.000084,0.007088,0.033968,0.076728,0.016350,0.003168,0.017293,0.028359,0.037432,...,0.000010,8.686617e-06,1.700473e-05,4.535204e-06,1.458934e-06,4.466640e-07,7.579326e-08,6.435894e-09,1,1
4,0.000003,0.000046,0.001499,0.013960,0.026140,0.028661,0.003578,0.009999,0.021098,0.016831,...,0.000002,3.709901e-06,1.328471e-06,3.481364e-07,1.555228e-07,5.434392e-08,1.003702e-08,8.769763e-10,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,0.000002,0.000019,0.000075,0.000058,0.000013,0.000023,0.000395,0.013317,0.032795,0.075172,...,0.000172,6.577667e-05,3.777974e-05,9.589142e-06,4.696875e-06,1.543086e-06,3.007084e-07,4.978052e-08,0,8
1436,0.000064,0.000131,0.000262,0.000223,0.000111,0.000136,0.002221,0.046049,0.076049,0.219650,...,0.000594,3.106455e-04,1.621839e-04,1.005938e-04,3.059142e-05,9.542340e-06,1.252834e-06,1.972792e-07,0,8
1437,0.000048,0.000030,0.000399,0.000541,0.000190,0.000160,0.000925,0.031382,0.103851,0.084082,...,0.000687,3.384231e-04,1.500613e-04,7.867455e-05,2.148737e-05,6.161907e-06,6.787151e-07,5.262246e-08,0,8
1438,0.003726,0.001217,0.000471,0.000263,0.000234,0.000296,0.001027,0.007759,0.036937,0.147467,...,0.003073,1.784341e-03,1.175709e-03,5.314132e-04,2.158249e-04,6.307891e-05,1.148216e-05,1.022560e-06,0,8


In [20]:
data.shape

(1440, 130)